# Delta Practice

### File Read In

In [62]:
import pandas as pd

filepath1 = r'/Users/cartersocha/Downloads/instgramHashtagCounts.xlsx'
instaHashtagDf = pd.read_excel(filepath1)

filepath2 = r'/Users/cartersocha/Downloads/tweetCountTest.xlsx'
dailyTweetDf = pd.read_excel(filepath2)

filepath3 = r'/Users/cartersocha/Desktop/ReleaseData.xlsx'
releaseDf = pd.read_excel(filepath3, "ShowInfoEndStart")

filepath4 = r'/Users/cartersocha/Downloads/instgramAccountCounts.xlsx'
igAccountDf = pd.read_excel(filepath4)

filepath5 = r'/Users/cartersocha/Downloads/redditCountTest.xlsx'
redditSubsDf = pd.read_excel(filepath5)

filepath6 = r'/Users/cartersocha/Downloads/redditCommentData.xlsx'
redditCommentsDf = pd.read_excel(filepath6)

filepath8 = r'/Users/cartersocha/Desktop/ReleaseData.xlsx'
releaseDateDf = pd.read_excel(filepath8, "ReleaseDateData")

filepath10 = r'/Users/cartersocha/Downloads/googleDataset.csv'
googleDailyData = pd.read_csv(filepath10)

### Functions

In [63]:
def DateTimeConvert(dateDf, dateColumn):
    dateDf[dateColumn] = pd.to_datetime(dateDf[dateColumn])  

    return dateDf

In [64]:
def DiffMaker(fillnaDf, valueColumn, dateColumn):
    fillnaDf.sort_values(['TvShow', dateColumn], inplace=True)

    fillnaDf['diffs'] = fillnaDf.groupby(['TvShow'])[valueColumn].transform(lambda x: x.diff()).fillna(0)

    fillnaDf.sort_index(inplace=True)

    return fillnaDf

In [65]:
def RemoveData(release,showDf):

    bigDf = pd.DataFrame()

    for i in range(len(release)):
        show = release['TvShow'][i]
        firstDate = release['Release Date'][i]
        secondDate = release['90DayDate'][i]

        smallDf = showDf[showDf['TvShow'] == show]

        newdf = smallDf[smallDf['RunDate'].between(firstDate, secondDate)]

        bigDf = bigDf.append(newdf,ignore_index=True)

    return bigDf

In [66]:
def MergeDfs(mainDf, secondDf, columnName):

    merged = pd.merge(mainDf,secondDf, how='outer', on=columnName)
    return merged

In [67]:
import numpy as np

def NegativeDiffs(diffDf,columnName):

    diffDf['zeroedDiffs'] = np.where((diffDf[columnName] < 0), 0, diffDf[columnName])

    return diffDf

In [68]:
def SumSocialMedia(sumDf,columnName):
    # check to see if this is episode count or generic summarization
    if type(columnName) != list:
        summarizedDf = sumDf.groupby(columnName, as_index=False).sum()
        summarizedDf = pd.DataFrame(summarizedDf)
    
    summarizedDf = sumDf.groupby(columnName, as_index=False).count()
    summarizedDf = pd.DataFrame(summarizedDf)
    
    return summarizedDf

### Data Transformations

##### Release Data

In [69]:
releaseDf['90DayDate'] = releaseDf['Release Date'] + pd.DateOffset(days=90)

releaseDf = DateTimeConvert(releaseDf,'90DayDate')
releaseDf = DateTimeConvert(releaseDf,'Release Date')

##### Instagram Hashtag Data

In [70]:
instaHashtagDf = DateTimeConvert(instaHashtagDf,'RunDate')

instaHashtagDf = DiffMaker(instaHashtagDf,'HashtagValue','RunDate')

instaHashDf = RemoveData(releaseDf, instaHashtagDf)
instaHashDf = NegativeDiffs(instaHashDf, 'diffs')
instaHashDf['SocialMediaSource'] = 'InstagramHashtag'
instaHashDf['SocialMediaValue'] = instaHashDf['zeroedDiffs']

#iHashtagDf = SumSocialMedia(instaHashDf, 'TvShow')

#iHashtagDf['SocialMediaValue'] = iHashtagDf['zeroedDiffs']
#iHashtagDf['SocialMediaSource'] = 'InstagramHashtag'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_14855/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Twitter Data

In [71]:
dailyTweetDf = DateTimeConvert(dailyTweetDf,'RunDate')
dailyTweetDf = RemoveData(releaseDf, dailyTweetDf)
dailyTweetDf['SocialMediaValue'] = dailyTweetDf['TweetCount'] + dailyTweetDf['RetweetCount']
dailyTweetDf['SocialMediaSource'] = 'Tweets'

#iTweetCountDf = SumSocialMedia(dailyTweetDf, 'TvShow')
#iTweetCountDf['SocialMediaSource'] = 'Tweets'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_14855/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Reddit Comment Data

In [72]:
redditCommentsDf['TvShow'] = redditCommentsDf['Show']
redditCommentsDf['RunDate'] = redditCommentsDf['runDate']
redditCommentsDf = DateTimeConvert(redditCommentsDf,'RunDate')

redCommentDf = RemoveData(releaseDf, redditCommentsDf)
redCommentDf['SocialMediaValue'] = redCommentDf['NumComments'] + redCommentDf['score']

redCommentDf = redCommentDf[['TvShow','RunDate','SocialMediaValue']]
iRedCommentDf = SumSocialMedia(redCommentDf, ['TvShow','RunDate'])
iRedCommentDf['SocialMediaSource'] = 'RedditComments'

#iRedCommentDf['SocialMediaValue'] = redCommentDf['NumComments'] + redCommentDf['score']
#iRedCommentDf['SocialMediaSource'] = 'RedditComments'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_14855/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Instagram Account Data

In [73]:
igAccountDf = DateTimeConvert(igAccountDf,'RunDate')

igAccountDf = DiffMaker(igAccountDf,'IgAccountCounts','RunDate')


instaAccDf = RemoveData(releaseDf, igAccountDf)
instaAccDf = NegativeDiffs(instaAccDf, 'diffs')
instaAccDf['SocialMediaSource'] = 'InstagramAccount'
instaAccDf['SocialMediaValue'] = instaAccDf['zeroedDiffs']

#iAccountDf = SumSocialMedia(instaAccDf, 'TvShow')

#iAccountDf['SocialMediaValue'] = iAccountDf['zeroedDiffs']
#iAccountDf['SocialMediaSource'] = 'InstagramAccount'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_14855/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Reddit Subscriber Data

In [74]:
redditSubsDf = DateTimeConvert(redditSubsDf,'RunDate')

redditSubsDf = DiffMaker(redditSubsDf,'RedditSubs','RunDate')

redSubDf = RemoveData(releaseDf, redditSubsDf)
redSubDf = NegativeDiffs(redSubDf, 'diffs')
redSubDf['SocialMediaSource'] = 'RedditSubscribers'
redSubDf['SocialMediaValue'] = redSubDf['zeroedDiffs']

#iRedSubDf = SumSocialMedia(redSubDf, 'TvShow')
#iRedSubDf['SocialMediaValue'] = iRedSubDf['zeroedDiffs']
#iRedSubDf['SocialMediaSource'] = 'RedditSubscribers'

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_14855/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Google Data

googleTrendsDf = DateTimeConvert(googleTrendsDf,'RunDate')

googDf = RemoveData(releaseDf, googleTrendsDf)
googDf = NegativeDiffs(googDf, 'GoogleValue')
googDf['SocialMediaSource'] = 'Google'
googDf['SocialMediaValue'] = googDf['zeroedDiffs']

In [75]:
googleDailyData = DateTimeConvert(googleDailyData,'RunDate')
googDf1 = RemoveData(releaseDf, googleDailyData)
googDf1 = NegativeDiffs(googDf1, 'GoogleValue')
googDf1['SocialMediaSource'] = 'Google'
googDf1['SocialMediaValue'] = googDf1['zeroedDiffs']

/var/folders/7v/2b53j97121jg00pcthbt6kz80000gn/T/ipykernel_14855/2781502079.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  bigDf = bigDf.append(newdf,ignore_index=True)


##### Release Date Data

In [76]:
releaseDateDf = DateTimeConvert(releaseDateDf,'EpisodeReleaseDate')
iReleaseData = SumSocialMedia(releaseDateDf, ['TvShow','EpisodeReleaseDate'])
iReleaseData['DailyReleaseCount'] = iReleaseData['EpisodeNumber']
iReleaseData['RunDate'] = iReleaseData['EpisodeReleaseDate']

##### Join the Data

##### Time Value Datasets

In [77]:
instaAccDf = instaAccDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]#,'diffs']]
redSubDf = redSubDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]#,'diffs']]
dailyTweetDf = dailyTweetDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
instaHashDf = instaHashDf[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]#,'diffs']]
googDf1 = googDf1[['TvShow','RunDate','SocialMediaValue','SocialMediaSource']]
# reddit comment data is combined elsewhere
iReleaseData = iReleaseData[['TvShow','RunDate','DailyReleaseCount']]

In [78]:
uberDf2 = pd.concat([instaAccDf,redSubDf,dailyTweetDf,instaHashDf,googDf1,iRedCommentDf])

uberDf3 = MergeDfs(uberDf2, releaseDf,'TvShow')
uberDf3

uberDf3 = MergeDfs(uberDf3, iReleaseData,['TvShow','RunDate'])
uberDf3 = uberDf3[uberDf3['Ignore'] == 1]
uberDf3 = uberDf3.fillna(0)
uberDf3.head()

,TvShow,RunDate,SocialMediaValue,SocialMediaSource,Stream,EpisodeCount,SeasonNumber,ReleaseCadence,Release Date,Ignore,ShowStatus,90DayDate,DailyReleaseCount
0,MoonKnight,2022-03-30,2481.0,InstagramAccount,Disney,6.0,1.0,Weekly,2022-03-30,1.0,Completed,2022-06-28,1.0
1,MoonKnight,2022-03-30,134.0,RedditSubscribers,Disney,6.0,1.0,Weekly,2022-03-30,1.0,Completed,2022-06-28,1.0
2,MoonKnight,2022-03-30,165358.0,Tweets,Disney,6.0,1.0,Weekly,2022-03-30,1.0,Completed,2022-06-28,1.0
3,MoonKnight,2022-03-30,2913.0,InstagramHashtag,Disney,6.0,1.0,Weekly,2022-03-30,1.0,Completed,2022-06-28,1.0
4,MoonKnight,2022-03-30,64.0,Google,Disney,6.0,1.0,Weekly,2022-03-30,1.0,Completed,2022-06-28,1.0


### Output - individuals & uber dataset

filepath2 = r'/Users/cartersocha/Downloads/uberDataset.csv'

uberDf3.to_csv(filepath2) 